## Ejecutamos el script para obtener las transcripciones

#### Importar transcripcines

In [6]:
import os

# Ejecutamos el script crear_transcripciones.py
%run /home/jovyan/work/scripts/crear_transcripciones.py

# Para listar archivos en el directorio actual
# print("Archivos en el directorio actual:")
# print(os.listdir('/home/jovyan/work'))


Transcripciones extraidas correctamente y guardadas en la carpeta 'transcripciones' (en la carpeta de notebooks).


## Limpieza de datos

In [7]:
import os
import re
from bs4 import BeautifulSoup
from num2words import num2words

def clean_text(text):
    # Eliminar etiquetas HTML
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    # Eliminar los sonidos externos entre corchetes
    text = re.sub(r'\[.*?\]', '', text)
    # Eliminar los sonidos externos marcados con *
    text = re.sub(r'\*.*?\*', '', text)
    # Eliminar los sonidos externos entre paréntesis
    text = re.sub(r'\(.*?\)', '', text)
    # Convertir números a texto en español
    text = re.sub(r'\d+', lambda x: num2words(int(x.group()), lang='es'), text)
    # Eliminar caracteres especiales
    text = re.sub(r'[^A-Za-záéíóúüñÁÉÍÓÚÜÑ\s]', '', text)
    # Eliminar múltiples espacios en blanco
    text = re.sub(r'\s+', ' ', text).strip()
    return text

input_directory = '/home/jovyan/work/data/correccion_gramatical'
output_directory = '/home/jovyan/work/data/cleaned'

# Guardamos los archivos
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    
for filename in os.listdir(input_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            cleaned_text = clean_text(text)
            with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
                out_file.write(cleaned_text)


## Corrección ortográfica y gramatical

In [13]:
import os
import requests
import time
from transformers import GPT2LMHeadModel, GPT2Tokenizer

GPT = False

# Directorios de entrada y salida
input_directory = '/home/jovyan/work/data/transcripciones'
output_directory = '/home/jovyan/work/data/correccion_gramatical'

# Crear el directorio de salida si no existe
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Limitamos el texto a 500 caracteres
def limit_text(text, max_length=250):
    return text[:max_length]

# Función para corregir texto usando GPT
if GPT:
    model_name = 'gpt2-large'
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    def correct_text_gpt(text):
        inputs = tokenizer.encode("Correct this sentence: " + text, return_tensors="pt")
        outputs = model.generate(inputs, max_length=500, num_return_sequences=1)
        corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return corrected_text

# Función para corregir texto usando la API de LanguageTool
else:
    def correct_text_api(text, language='es', retries=3, backoff=2):
        url = 'https://api.languagetool.org/v2/check'
        data = {
            'text': text,
            'language': language
        }
        
        for attempt in range(retries):
            try:
                response = requests.post(url, data=data)
                response.raise_for_status()  # Lanza una excepción para códigos de estado HTTP 4xx/5xx
                return response.json()
            except requests.RequestException as e:
                print(f"Error en la solicitud a LanguageTool API: {e}. Intento {attempt + 1} de {retries}.")
                if attempt < retries - 1:
                    time.sleep(backoff ** attempt)  # Exponencial backoff
                else:
                    raise

    # Función para aplicar correcciones preservando la estructura del texto
    def apply_corrections(text, matches):
        corrections = []
        for match in matches['matches']:
            if match['replacements']:
                correction = {
                    'offset': match['offset'],
                    'length': match['length'],
                    'replacement': match['replacements'][0]['value']
                }
                corrections.append(correction)
        
        corrections.sort(key=lambda x: x['offset'], reverse=True)
        
        for correction in corrections:
            start = correction['offset']
            end = start + correction['length']
            text = text[:start] + correction['replacement'] + text[end:]
        
        return text

# Procesar los archivos en el directorio de entrada
for filename in os.listdir(input_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            # Limitar el texto a 500 caracteres
            text = limit_text(text)
            try:
                if GPT:
                    corrected_text = correct_text_gpt(text)
                else:
                    matches = correct_text_api(text)
                    corrected_text = apply_corrections(text, matches)

                # Guardar el texto corregido en el directorio de salida
                with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
                    out_file.write(corrected_text)
                print(f"Archivo '{filename}' corregido y guardado.")
            except Exception as e:
                print(f"No se pudo corregir el archivo '{filename}': {e}")

print("Corrección gramatical completada.")


Archivo 'El Rubius_Q0xVOwNmx9w.txt' corregido y guardado.
Archivo 'Yuya_wd_MQ3os0M0.txt' corregido y guardado.
Archivo 'Luisito Comunica_2at1uc0LfQY.txt' corregido y guardado.
Archivo 'Luisito Comunica_w-pIBtP21vc.txt' corregido y guardado.
Archivo 'El Rubius_bXSIpq11MfE.txt' corregido y guardado.
Archivo 'Yuya_ldATTcJpIpE.txt' corregido y guardado.
Archivo 'DrossRotzank_6aswzDXxj7U.txt' corregido y guardado.
Archivo 'DrossRotzank_q7Lo6SLfapg.txt' corregido y guardado.
Corrección gramatical completada.


## Tokenizacion

In [14]:
import os
import spacy

# spaCy en español
nlp = spacy.load("es_core_news_sm")

def tokenize(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

# input_directory = '/home/jovyan/work/data/corrected'
input_directory = '/home/jovyan/work/data/normalized'
output_directory = '/home/jovyan/work/data/tokenized'

# Guardamos los archivos tokenizados
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(input_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            tokens = tokenize(text)
            with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
                out_file.write(' '.join(tokens))


## Análisis de Resultados
### Comparación de Textos Originales y Corregidos

In [15]:
import os

# Función para leer los caracteres de un archivo desde un índice n hasta m
def read_chars_from_n_to_m(file_path, n, m):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        return text[n:m]

# Función para mostrar la comparación entre el texto original y el corregido
def compare_texts(original, corrected):
    print(f"Original: {original}")
    print(f"Corrected: {corrected}")
    print("=" * 50)

# Función principal para comparar textos en archivos
def compare_files(original_directory, corrected_directory, start_char, end_char):
    # Obtener los nombres de los archivos en el directorio original
    file_names = [f for f in os.listdir(original_directory) if f.endswith('.txt')]

    # Aplicar la comparación a los archivos de texto
    for file_name in file_names:
        original_path = os.path.join(original_directory, file_name)
        corrected_path = os.path.join(corrected_directory, file_name)
        
        # Verificar que el archivo corregido exista
        if os.path.exists(corrected_path):
            original_text = read_chars_from_n_to_m(original_path, start_char, end_char)
            corrected_text = read_chars_from_n_to_m(corrected_path, start_char, end_char)
            
            # Comparar los textos
            compare_texts(original_text, corrected_text)
        else:
            print(f"Archivo corregido no encontrado para: {file_name}")

# Directorios de transcripciones originales y corregidas
original_directory = '/home/jovyan/work/data/transcripciones'
corrected_directory = '/home/jovyan/work/data/tokenized'

# Índices de caracteres a comparar
start_char = 100
end_char = 350

# Llamada a la función principal
compare_files(original_directory, corrected_directory, start_char, end_char)


Original: ballo vale eh No hay tiempo para intros hay que buscar a a Don Ramón que está perdido en el mar eh Espero que siga vivo por favor que esté vivo no sé por qué me he jugado la vida por un barco la verdad cuando es casi gratis de hacer pero vale dónde m
Corrected: ballo vale eh no hay tiempo para antros hay que buscar a don ramón que está perdido en el mar eh espero que siga vivo por favor que esté vivo no sé por qué me he jugado la vida por un barco la verdad cuando es casi gratis de hacer pero vale dónde me 
Original:  ahora podrás como es tan diario yo soy yo ya juegan amigos a díaz ha llegado el momento de volvernos a encontrar ahora sí que no procrastinar realmente no podía usar no me acomodaba pasaron muchas cosas ha sido hermoso y caótico también esto que ven
Corrected: uegan amigos a díaz ha llegado el momento de volvernos a encontrar ahora sí que no procrastinar realmente no podía usar no me acomodaba pasaron muchas cosas ha sido hermoso y caótico también esto que ven a